# Iterative Prisoner's Dilemma

### Description
In this notebook is presented the code in order to resolve the Iterative Prisoner's Dilemma (IPD).

### Player and Strategy class
For our goal we defined a class player with different methods that aim to store the history of moves and play against other players. We defined a class strategy wchich methods generate strategy given the number of players.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
#from mgen import generatePayoffMatrix

# Define global variables to have groundtruth about strategies
COOPERATE = 0
DEFECT = 1

NICE = 0
IND  = 50
BAD  = 100

TFT  = -1
TF2T = -2
GRT  = -3
PRBL = -10 # just placeholders
PRBH = -11

class Strategy:
    """Abstract Strategy class to derive other."""
    TOT_STRAT = 8

    def __str__(self):
        return "Base"

    def get(self):
        pass

    @staticmethod
    def generatePlayers(num_players, replace=False, fixed=False):
        """Generates a set of players with random strategies."""
        str_choices = [NICE, BAD, IND, TFT, TF2T, GRT, PRBL, PRBH]
        k = np.random.choice(str_choices, num_players, replace=replace)

        # substitute with useful values if needed
        maskL = k==PRBL # 1 to 49 exclude nice, indifferent
        k[maskL] = np.repeat(25, maskL.sum()) if fixed else np.random.choice(49, size=maskL.sum(), replace=replace) + 1
        maskH = k==PRBH # 51 to 99 exclude bad, indifferent
        k[maskH] = np.repeat(75, maskH.sum()) if fixed else np.random.choice(49, size=maskH.sum(), replace=replace) + 51

        return k

class Player(object):
    """Class to describe a player with strategy and history."""

    def __str__(self):
        return "Player with strategy: {}".format(self.s)

    # optional: use M = generatePayoffMatrix()
    def __init__(self, k=0, M=np.array([[3,0],[5,1]]), changing=False):
        self.M1 = M
        self.M2 = M.T
        if changing:
            self.c = np.random.uniform(0,1) # probability to change strategy at the end of the round
        else:
            self.c = 0
        self.s = self.get_strategy(k)
        self.clear_history()

    def play_iter(self, opponent, num_iter):
        """Plays the game against an opponent num_iter times."""
        for _ in range(num_iter):
            self.play(opponent)

    def play(self, opponent):
        """Plays the game against an opponent."""
        action1 = self.act(opponent)
        action2 = opponent.act(self)

        self.update(action1, action2, False)
        if opponent.s != self.s:
            opponent.update(action1, action2, True)

    def act(self, opponent):
        """Gets the action based on the strategy."""
        if type(self.s) == ProbStrategy:
            return self.s.get()
        elif type(self.s) == TitForTat or type(self.s) == GrimTrigger:
            if len(opponent.playedHist) > 0: # at least 1
                return self.s.get(opponent.playedHist[-1]) # pass opponent's move
            return COOPERATE
        elif type(self.s) == TitFor2Tat:
            if len(opponent.playedHist) > 1: # at least 2
                return self.s.get([opponent.playedHist[-2],opponent.playedHist[-1]]) # pass opponent's previous two moves
            return COOPERATE

    def update(self, action1, action2, opponent):
        """Updates the state based on the actions and if is opponent."""
        if opponent:
            self.payoffHist.append(self.M2[action1,action2])
            self.playedHist.append(action2)
        else:
            self.payoffHist.append(self.M1[action1,action2])
            self.playedHist.append(action1)

    def get_strategy(self, k):
        """Gets the strategy object given the id."""
        if k >= NICE and k <= BAD:
            return ProbStrategy(k)
        elif k == TFT:
            return TitForTat()
        elif k == TF2T:
            return TitFor2Tat()
        elif k == GRT:
            return GrimTrigger()

    def clear_history(self):
        """Clears all history of the player."""
        self.payoffHist = []
        self.playedHist = []

class MultiPlayer(Player):
    """Class to describe multiple players with strategy and history."""

    def __init__(self, k, changing=False):
        Player.__init__(self, k, changing=changing)

        # save results for multiple rounds played by user
        # this way we can save all the results from the tournament
        self.prevPlayedHist = []
        self.prevOpponent = []
        self.results = []
        self.changing = changing

    # TODO:  Un parametro del nostro grado di cooperazione potrebbe essere la percentuale sul
    # totale di volte che una strategia ha cooperato durante il torneo precedente.
    @staticmethod
    def change_strategy(players):
        """Change the players' strategy randomly."""
        c_b = 0
        c_g = 0
        k_strategies = Strategy.generatePlayers(len(players)*3, replace=(len(players)*3>Strategy.TOT_STRAT))
        for i in range(len(players)):
            if i < len(players)/2:
                ##TODO TUNE THIS, maybe refer to the position
                print("Strategy that gives me good results - reinforce my type")
                print("BEFORE s {}, c {}".format(players[i].s, players[i].c))
                if players[i].s.id > IND:
                    players[i].c = (players[i].c + players[i].c**2)/2
                else:
                    players[i].c = (players[i].c + players[i].c**0.5)/2
                print("AFTER s {}, c {}".format(players[i].s, players[i].c))
            elif i > len(players)/2:
                print("Strategy that gives me bad results - try to change my type in the opposite direction")
                print("BEFORE s {}, c {}".format(players[i].s, players[i].c))
                if players[i].s.id > IND:
                    players[i].c = (players[i].c + players[i].c**0.5)/2
                else:
                    players[i].c = (players[i].c + players[i].c**2)/2
                print("AFTER s {}, c {}".format(players[i].s, players[i].c))
            else:
                print("In the middle, not changing my type s {}, c {}".format(players[i].s, players[i].c))

            #MUTATION PROBABILITY RELATED TO RESULTS
            if np.random.uniform(0,1) < i/len(players):
                #If lower I am tempted to cooperate
                if np.random.uniform(0,1) > players[i].c:
                    print("I am going to a less cooperative behaviour")
                    if players[i].s.id < BAD:
                        s_next = players[i].random_strategy(k_strategies)
                        c_b += 1
                        while str(s_next) == str(players[i].s) or (s_next.id < players[i].s.id or s_next.id < IND):
                            s_next = players[i].random_strategy(k_strategies)
                        players[i].s = s_next
                    print("After change of type I am {}\n\n".format(players[i].s))
                else:
                    print("I am going to a more cooperative behaviour")
                    if players[i].s.id > GRT:
                        s_next = players[i].random_strategy(k_strategies)
                        c_g += 1
                        while str(s_next) == str(players[i].s) or (s_next.id > players[i].s.id or s_next.id > IND):
                            s_next = players[i].random_strategy(k_strategies)
                        players[i].s = s_next
                    print("After change of type I am {}\n\n".format(players[i].s))
            print("\n\n")
        return players, c_b, c_g

    def random_strategy(self, k_list):
        """Generate random strategy object."""
        return self.get_strategy(np.random.choice(k_list))

    def play_iter(self, opponent, num_iter):
        """Plays the game against an opponent num_iter times."""
        Player.play_iter(self, opponent, num_iter)

        # save history for both players
        self.prevPlayedHist.append(self.playedHist)
        if self.s != opponent.s:
            opponent.prevPlayedHist.append(opponent.playedHist)

        self.prevOpponent.append(opponent)
        if self.s != opponent.s:
            opponent.prevOpponent.append(self)

        # check the sum of rewards to detect the winner
        self.winner(opponent)

    def winner(self, opponent):
        """Saves the total payoff of the player."""
        self.results.append(np.sum(self.payoffHist))
        if opponent.s != self.s:
            opponent.results.append(np.sum(opponent.payoffHist))

    def get_points(self):
        """Current gained points."""
        return np.cumsum(self.results)

    def get_coop_def_count(self):
        """Number of times the user cooperated and defected."""
        cooperate_count = defect_count = 0
        for hist in self.prevPlayedHist:
            cooperate_count += hist.count(COOPERATE)
            defect_count += hist.count(DEFECT)
        return cooperate_count, defect_count

class ProbStrategy(Strategy):
    """Strategy class when probability is used.
    Spans from always nice to always bad players"""

    def __init__(self, k):
        # default value is to cooperate in case of wrong k
        self.k = k if k>=NICE and k<=BAD else NICE
        self.id = k

    def get(self):
        num = np.random.randint(0,100)
        return COOPERATE if num >= self.k else DEFECT

    def __str__(self):
        if (self.k == NICE):
            return "Nice"
        elif (self.k == BAD):
            return "Bad"
        elif (self.k > IND):
            return "MainlyBad (k={})".format(self.k)
        elif (self.k < IND):
            return "MainlyNice (k={})".format(self.k)
        else:
            return "Indifferent"

class TitForTat(Strategy):
    """Plays opponent's last move."""

    def __init__(self):
        self.id = TFT

    def __str__(self):
        return "TitForTat"

    def get(self, last_move=None):
        if last_move == None:
            return COOPERATE # first time
        return last_move # repeat past opponent move

class TitFor2Tat(TitForTat):
    """Defects only if the opponent has defected last two times."""

    def __init__(self):
        self.id = TF2T

    def __str__(self):
        return "TitFor2Tat"

    def get(self, last_moves=None):
        if last_moves == None:
            return COOPERATE
        return (last_moves[0] and last_moves[1])
        # COOPERATE = 0, DEFECT = 1. If both 1 only case to have DEFECT as output

class GrimTrigger(Strategy):
    """Cooperate at first, if opponent defects once then always defect."""

    def __init__(self):
        self.id = GRT
        self.triggered = False

    def __str__(self):
        return "GrimTrigger"

    def get(self, last_move=None):
        if not self.triggered and last_move == DEFECT:
            self.triggered = True

        if self.triggered:
            return DEFECT
        return COOPERATE


### Matrix Generator
A simple function that generate and return a payoff matrix M.

In [ ]:
import numpy as np
def generatePayoffMatrix(max=20, use_positive=True):
    """Generates a valid payoff matrix for the IPD problem."""
    # M = [R,S;T,P] where T>R>P>S, 2R>T+S

    ok = False
    while not ok:
        if use_positive:
            P = np.random.randint(0,max)
            S = np.random.randint(0,P)
        else:
            P = np.random.randint(-max,max)
            S = np.random.randint(-max,P)
        R = np.random.randint(P,max)
        T = 2*R-S-1
        if T>R and R>P and P>S and 2*R>T+S:
            ok = True
    return np.array([[R,S],[T,P]])

### Iterative prisoner's dilemma between two players
Below is implement a simple IPD between two players. We iterate the process in orther to find the mean and standar deviation of the final result for the match. 

We plot the history of the player's moves.

In [ ]:
def main():
    np.random.seed(100)
    
    SAVE_IMG = False

    NUM_ITER = 50
    NUM_PLAYERS = 8
    NUM_REPETITIONS = 100

    print("Testing {} iterations of 2-people IPD".format(NUM_ITER))

    # define k for strategy probabilities
    k_strategies = Strategy.generatePlayers(NUM_PLAYERS, replace=False)
    matches_df = pd.DataFrame() # all matches played
                 
    for first in range(NUM_PLAYERS):
        for second in range(first, NUM_PLAYERS):
            k1 = k_strategies[first]
            k2 = k_strategies[second]

            p1 = Player(k1)
            p2 = Player(k2)
            rew1 = np.zeros_like(NUM_ITER)
            rew2 = np.zeros_like(NUM_ITER)
            print("Evaluating {} - {}...".format(p1.s, p2.s))

            # repeat the match to get some statistics (mean and std)
            cum_results = { k1:[], k2:[] }
            for _ in range(NUM_REPETITIONS):
                p1.clear_history()
                p2.clear_history()
                p1.play_iter(p2, NUM_ITER)
            
                rew1 = np.cumsum(p1.payoffHist)
                rew2 = np.cumsum(p2.payoffHist)
                cum_results[k1].append(rew1[-1])
                cum_results[k2].append(rew2[-1])
                
            # table
            df = pd.DataFrame(
                [[p1.s, p2.s, 
                np.mean(cum_results[k1]), np.std(cum_results[k1]), 
                np.mean(cum_results[k2]), np.std(cum_results[k2])]],
                columns=['p1','p2','p1-avg','p1-std', 'p2-avg', 'p2-std']
            )
            matches_df = matches_df.append(df)
            

            # boxplots for 100 matches -> A vs B
            plt.boxplot([cum_results[k1], cum_results[k2]])
            plt.xticks([1, 2], [p1.s, p2.s])
            plt.ylabel('Reward')
            plt.title("Meand and std for {} iterations".format(NUM_REPETITIONS))
            if SAVE_IMG:
                plt.savefig('../img/ipd2p/ipd2p-boxplot-{}-{}.eps'.format(str(p1.s).replace(" ",""),str(p2.s).replace(" ","")),format='eps',bbox_inches='tight')
                plt.close()
            else:
                plt.show()
            
            # plot cumulative rewards
            # show only the last iteration's plot
            plt.figure(figsize=(12,5))    
            plt.plot(rew1)
            plt.plot(rew2)
            for i in range(rew1.size):
                if p1.playedHist[i] == COOPERATE:
                    plt.plot(i, rew1[i], 'bx', markersize=8)
                else:
                    plt.plot(i, rew1[i], 'rx', markersize=8)

                if p2.playedHist[i] == COOPERATE:
                    plt.plot(i, rew2[i], 'bo', markersize=5)
                else:
                    plt.plot(i, rew2[i], 'ro', markersize=5)

            plt.title("2 pl. game: {} - {}".format(p1.s,p2.s))
            plt.xlabel('Iteration')
            plt.ylabel('Cum. reward')
            plt.legend(handles=[
                Line2D([0], [0], color='w', marker='x', label='P.1 Defect', markeredgecolor='r'), 
                Line2D([0], [0], color='w', marker='x', label='P.1 Cooperate', markeredgecolor='b'),      
                Line2D([0], [0], color='w', marker='o', label='P.2 Defect', markerfacecolor='r'), 
                Line2D([0], [0], color='w', marker='o', label='P.2 Cooperate', markerfacecolor='b')
            ])

            if SAVE_IMG:
                plt.savefig('../img/ipd2p/ipd2p-rewards-{}-{}.eps'.format(str(p1.s).replace(" ",""),str(p2.s).replace(" ","")),format='eps',bbox_inches='tight')
                plt.close()
            else:
                plt.show()

    pd.set_option('precision', 2)
    print(matches_df.to_latex(index=False))

if __name__ == "__main__":
    main()

### Multiple Players Iterative Prisoner's Dilemma (MPIPD)
In this section is implemented the MPIPD. Given a number of players, we perform MPIPD in a round-robin scheme. In other words we create a tournament between multiple players where each player play against all the other participants. We iterate the tournament multiple times and we show the results in boxplots. 

In [ ]:
SAVE_IMG = False

def IPDRoundRobin(players, num_iter, against_itself=False, plot=False):
    """Round Robin tournament."""
    n = len(players)

    # each player plays against another in a round robin scheme
    if plot:
        plt.figure(figsize=(12,5))

    p = {obj:[0] * num_iter for obj in players}

    for (i, p1) in zip(np.arange(n), players):
        start = i if against_itself else i+1
        for (j, p2) in zip(np.arange(start, n), players[start:]):
            p1.clear_history()
            p2.clear_history()
            p1.play_iter(p2, num_iter)
            if plot:
                p[p1] += np.cumsum(p1.payoffHist)
                p[p2] += np.cumsum(p2.payoffHist)

    if plot:
        for i in p:
            plt.plot(p[i], label=i.s)
            plt.xlabel('Iteration')
            plt.ylabel('Cum. reward')
        plt.title("Evolution of the game")
        plt.legend(bbox_to_anchor=(1,1))
        if SAVE_IMG: # TODO we save images in ipdmp dir but this method is called also by other scripts
            plt.savefig('../img/ipdmp/ipdmp-evolution-of-game-{}.eps'.format(len(p)),format='eps',bbox_inches='tight')
            plt.close()

    # calculate ranking and matches dataframes
    # has to be done after the tournament
    ranking_df = pd.DataFrame() # all points gained by players
    matches_df = pd.DataFrame() # all matches played sorted by time

    for (i, p) in zip(np.arange(n), players):
        points = p.get_points()
        cooperate_count, defect_count = p.get_coop_def_count()

        df = pd.DataFrame(
            [[p.s, int(points[-1]), cooperate_count, defect_count, p, p.s.id]],
            columns=['Player','points', 'cooperate_count', 'defect_count', 'rrp', 'labels']
        )
        ranking_df = ranking_df.append(df)
        # ranking_df = ranking_df.sort_values(['points'], ascending=False)

        for j in range(i, len(p.results)):
            # can now access any property from p1 or p2 for plots
            # each match can be explored
            df = pd.DataFrame(
                [[p.s, p.prevOpponent[j].s, p.results[j], p.prevOpponent[j].results[i]]],
                columns=['p1','p2','p1-score','p2-score']
            )
            matches_df = matches_df.append(df)

    players = np.array(ranking_df.sort_values(['points'], ascending=False)['rrp'])
    ranking_df = ranking_df.drop(columns=['rrp']).reset_index(drop=True)
    return players, ranking_df, matches_df

def main():
    np.random.seed(100)
    pd.set_option('display.max_columns', None)

    NUM_ITER = 50
    NUM_PLAYERS = 50
    NUM_REPETITIONS = 10
    print("Testing round-robin tournament with {}-people".format(NUM_PLAYERS))

    # define k for strategy probabilities
    k_strategies = Strategy.generatePlayers(NUM_PLAYERS, replace=(NUM_PLAYERS > Strategy.TOT_STRAT))

    repeated_players = []
    for i in range(NUM_REPETITIONS):
        # initialize players with given strategies
        players = np.array([MultiPlayer(k) for k in k_strategies])

        players, ranking_df, matches_df = IPDRoundRobin(players, NUM_ITER, plot=(i==(NUM_REPETITIONS-1))) # not against itself, plot last rep.

        repeated_players.append(players)
        repeated_ranking_df = repeated_ranking_df.append(ranking_df) if i!=0 else ranking_df
        # print(ranking_df.to_latex(index=False))
        # print(matches_df.to_latex(index=False))

    # print tables
    pd.set_option('precision', 2)

    group = repeated_ranking_df[['points', 'cooperate_count', 'defect_count']].groupby(repeated_ranking_df.index)
    group_mean = group.mean()
    group_mean.columns = [str(col) + '_mean' for col in group_mean.columns]
    group_std = group.std()
    group_std.columns = [str(col) + '_std' for col in group_std.columns]
    group_df = group_mean.merge(group_std, left_index=True, right_index=True, how='left')
    group_df['cooperation_perc'] = group_df['cooperate_count_mean']/(group_df['cooperate_count_mean']+group_df['defect_count_mean'])
    group_df['str'] = repeated_ranking_df['Player'][:NUM_PLAYERS]
    print(group_df.to_latex(index=False))

    # box plot of last match
    one_round_results = [p.results for p in players]
    one_round = pd.DataFrame(one_round_results).T
    meds = one_round.median().sort_values(ascending=False)
    one_round = one_round[meds.index]
    plt.figure(figsize=(12,5))
    one_round.boxplot()
    plt.xticks(np.arange(NUM_PLAYERS)+1, [players[p].s for p in meds.index], rotation=90)
    plt.suptitle('Mean and variance for each type vs the other players \n One complete round')
    plt.ylabel('Points')
    plt.xlabel('Player')
    if SAVE_IMG:
        plt.savefig('../img/ipdmp/ipdmp-boxplot-single-match-{}.eps'.format(NUM_PLAYERS),format='eps',bbox_inches='tight')
        plt.close()
    else:
        plt.show()

    # box plot of all points
    group_median = group.median().sort_values(by=['points'], ascending=False)
    temp_df = pd.DataFrame()
    for index in group_median.index:
        temp_df = temp_df.append(group.get_group(index))
    temp_df['index'] = np.repeat(np.arange(50), NUM_REPETITIONS)
    plt.figure(figsize=(12,5))
    temp_df.boxplot(column='points', by='index')
    plt.xticks(np.arange(NUM_PLAYERS)+1, group_df['str'][group_median.index], rotation=90)
    plt.suptitle(("Mean and variance for each type at the end of the tournament - {} repetitions").format(NUM_REPETITIONS))
    plt.ylabel('Points')
    plt.xlabel('Player')
    if SAVE_IMG:
        plt.savefig('../img/ipdmp/ipdmp-boxplot-final-points-{}.eps'.format(NUM_PLAYERS),format='eps',bbox_inches='tight')
        plt.close()
    else:
        plt.show()

if __name__ == "__main__":
    main()

### Repeated Multiple Players Iterative Prisoner's Dilemma (RMPIPD)
We iterate what done above, but at each iteration we increase the population implementing a given strategy depending on the results that strategy achieved in the previous iteration. We implement in two different ways:

1. #### Constant Population
    The population remains constant adding strategies that achive good results and removing strategies that do not perform well. 

2. #### Increase Population
    In this case the population increase. We start from a population with one player for each strategies and we only add strategies that perfom well in the previus iteration.

### 1. Constant Population

In [ ]:
def main():
    np.random.seed(100)
    pd.set_option('display.max_columns', None)
    pd.set_option('precision', 2)

    SAVE_IMG = False

    NUM_ITER = 100
    NUM_PLAYERS = 50
    PERCENTAGE = 0.3
    print("Testing repeated round-robin tournament with {}-people".format(NUM_PLAYERS))

    k_strategies = Strategy.generatePlayers(NUM_PLAYERS, replace=(NUM_PLAYERS>Strategy.TOT_STRAT))

    NUM_REPETITIONS = 0
    MAX_ALLOWED = 10
    repeated_players = []
    # strategies evolution
    unique, counts = np.unique(k_strategies, return_counts=True)
    strategies_df = pd.DataFrame([counts],columns=unique)

    while np.unique(k_strategies, return_counts=True)[1].max() < k_strategies.size*3/4 and NUM_REPETITIONS < MAX_ALLOWED:
        NUM_REPETITIONS += 1
        # initialize players with given strategies
        players = np.array([MultiPlayer(k) for k in k_strategies])
        
        players, ranking_df, matches_df = IPDRoundRobin(players, NUM_ITER) # no strategy change, not against itself
        repeated_players.append(players)

        # create strategies history
        unique, counts = np.unique(k_strategies, return_counts=True)
        df = pd.DataFrame([counts],columns=unique)
        strategies_df = strategies_df.append(df)

        # easy fix (depending on task)
        # add one winner strategy or multiple previous winners?
        for i in range(0,int(NUM_PLAYERS * PERCENTAGE)):
            k_strategies = np.append(k_strategies, players[i].s.id)
            k_strategies = np.delete(k_strategies,np.argmax(players[NUM_PLAYERS-i-1].s.id))
        #display(ranking_df)
        # display(matches_df)

    if np.unique(k_strategies, return_counts=True)[1].max() > k_strategies.size*3/4:
        print("Convergence speed of round-robin tournament is {} with {}-people".format(NUM_REPETITIONS, NUM_PLAYERS))
    else:
        print("Convergence not reached")
        
    # save plots
    strategies_df = strategies_df.rename(index=str,
        columns={-3: "TitForTwoTat", -2: "GrimTrigger", -1: "TitForTat", 0: "Nice", 100: "Bad", 50: "Indifferent"})
    for c in strategies_df.columns:
        if str.isdigit(str(c)):
            if c > 50:
                strategies_df = strategies_df.rename(index=str, columns={c: "MainlyBad (k={})".format(c)})
            else:
                strategies_df = strategies_df.rename(index=str, columns={c: "MainlyNice (k={})".format(c)})

    strategies_df.index = np.arange(strategies_df.index.size)
    strategies_df = strategies_df.fillna(0)
    print(strategies_df.to_latex(index=False))
    strategies_df.plot(figsize=(12,5))    
    plt.legend(ncol=int(len(strategies_df.columns)/10), bbox_to_anchor=(1, 1))
    plt.title('Strategies evolution')
    plt.ylabel('Number of strategies')
    plt.xlabel('Time')
    if SAVE_IMG:
        plt.savefig('../img/ripdmp-const/ripdmp-evolution-const-pop-{}.eps'.format(NUM_PLAYERS),format='eps',bbox_inches='tight')
        plt.close()
    else:
        plt.show()

    for (r, players) in zip(np.arange(NUM_REPETITIONS), repeated_players):
        plt.figure(figsize=(12,5))
        for p in players:
            points = p.get_points()
            plt.plot(points, label=p.s)
            plt.title("Multi pl. game: {}".format(NUM_PLAYERS))
            plt.xlabel('Match number')
            plt.ylabel('Points')

        plt.legend(ncol=int(NUM_PLAYERS/10), bbox_to_anchor=(1, 1))

        if SAVE_IMG:
            plt.savefig('../img/ripdmp-const/ripdmp-scores-const-pop-{}-r{}.eps'.format(NUM_PLAYERS, r),format='eps',bbox_inches='tight')
            plt.close()
        else:
            plt.show()
            
if __name__ == "__main__":
    main()

### 2. Increase Population son da cambiare le legende

### Repeated Multiple Players Iterative Prisoner's Dilemma where strategies are allowed to mutate
A parameter (gene) encode the attidue of an individual strategie to cooperate. The gene can mutate randomly.